<a id="section-one"></a>
# <div style="color:#fff;display:fill;border-radius:10px;background-color:#000000;text-align:left;letter-spacing:0.1px;overflow:hidden;padding:20px;color:white;overflow:hidden;margin:0;font-size:100%"> - | Notebook resume</div>

<p style="font-size:16px; font-family:verdana; line-height: 1.7em; margin-left:20px">  
Hello Kagglers, I created this notebook because I experienced a lot of lag when trying to work on this competition within the Kaggle notebooks, and found that it was caused by the size of the datasets, if the original datasets are deleted, the lag disappears. That's why I made this notebook that resizes the original dataset (train and test), and also splits it into folds (train) and saves them as parquet files, a much better and more efficient file type. For it to work properly you need to delete the original dataset and replace it with these files, after deleting the lag is gone. I hope this notebook has been useful to you. I've also made the datasets public in case you don't want to do this, just search the datasets for the train test lag solution playground or click here <a href="https://www.kaggle.com/maxdiazbattan/playground-train-test-kfold-parquet">[link]</a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage before optimization is: {:.2f} MB'.format(start_mem))
    print('Memory usage after optimization is:  {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

<p style="font-size:16px; font-family:verdana; line-height: 1.7em">   
This function it's from this notebook <a href="https://www.kaggle.com/rinnqd/reduce-memory-usage">[link]</a>, I had to used it because the memory usage it's very high with this particular dataset. All the credits to Firas, was a very clever solution.</p>

In [ ]:
skf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train['kfold'] = -1
y = train.claim
for fold, (train_idx, test_idx) in enumerate(skf.split(X=train, y=y)):
    train.loc[test_idx,'kfold'] = fold

In [ ]:
train = reduce_mem_usage(train)

In [ ]:
test = reduce_mem_usage(test)

In [ ]:
train.to_parquet('train_kfold_reduced.parquet', index=False)

In [ ]:
test.to_parquet('test_reduced.parquet', index=False)